In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

/home/aditya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aditya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aditya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aditya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
imdb_master = pd.read_csv("data/imdb_master.csv", nrows=50000)
stop_words = set(stopwords.words('english'))

In [ ]:
imdb_master

In [ ]:
def label2int(x):
    if x == 'neg':
        return 0
    elif x == 'pos':
        return 1

vfunc = np.vectorize(label2int)
imdb_master['label'] = vfunc(imdb_master.label.values)

In [ ]:
imdb_master['review'] = imdb_master.review.str.lower()
imdb_master['review'] = imdb_master.review.str.strip()
imdb_master['review'] = imdb_master.review.str.replace("[0-9!\"#$%&()*+,-./:;<=>?@[\]^_`´{|}~]"," ")
imdb_master['review'] = imdb_master.review.str.replace("'", "")

In [3]:
review = imdb_master.review.values.tolist()

In [6]:
tok_review = [i.split() for i in review] 

In [7]:
filter_tok_review = list()
for i in tok_review:
    temp_list = list()
    for j in i:
        if j not in stop_words:
            temp_list.append(j)
    filter_tok_review.append(temp_list)

In [8]:
len(filter_tok_review)  #tokenize review not containing the stop words

50000

In [ ]:
filename = 'imdb_master_embedding_word2vec_100k_100.txt'

In [ ]:
print(imdb_master.label.unique())

In [ ]:
embedding_index = dict()
f = open(filename)
for count,line in enumerate(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embedding_index[word] = coefs
f.close()

In [9]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(filter_tok_review)
sequences = tokenizer_obj.texts_to_sequences(filter_tok_review)

In [4]:
max_length = max([len(s.split()) for s in review])

In [10]:
max_length

2470

In [ ]:
word_index = tokenizer_obj.word_index
review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment = imdb_master.label.values

In [ ]:
#print(word_index)
print(len(review_pad))
print(len(review_pad[0]))
#print(sentiment)

In [ ]:
print('found {} unique tokens'.format(len(word_index)))
print('shape of review tensor:', review_pad.shape)
print('shape of sentiment tensor:', sentiment.shape)

In [ ]:
num_words = len(word_index) + 1
print(num_words)

embedding_matrix = np.zeros((num_words, 100))
#print(embedding_matrix)

for word, i in word_index.items():
    #print(word, i)
    if i > num_words:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
print(embedding_matrix[1])

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.models import model_from_json

Using TensorFlow backend.


In [ ]:
model = Sequential()
embedding_layer = Embedding(num_words, 100,
                            embeddings_initializer = Constant(embedding_matrix),
                            input_length = max_length,
                            trainable = False
                           )
model.add(embedding_layer)
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
VALIDATION_SPLIT = 0.2

indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT*review_pad.shape[0])
X_train_pad = review_pad[:-num_validation_samples]
Y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
Y_test = sentiment[-num_validation_samples:]

In [ ]:
model.fit(X_train_pad, Y_train, batch_size=128, epochs=2, validation_data=(X_test_pad, Y_test), verbose=1)

In [17]:
test_sample = ['i should have killed myself before watching this']  #sample for testing

for count,i in enumerate(test_sample):
    string = ''
    for j in i.split():
        if j not in stop_words:
            string = string + j + ' '
        test_sample[count] = string.strip()

test_sample_token = tokenizer_obj.texts_to_sequences(test_sample)
test_sample_token_pad = pad_sequences(test_sample_token, maxlen=2482)

In [18]:
result = model.predict(x=test_sample_token_pad)
print(result)

[[0.7665736]]


In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [14]:
# load json and create model
json_file = open('model.json', 'r')
model = json_file.read()
json_file.close()
model = model_from_json(model)
# load weights into new model
model.load_weights("model.h5")
print("model loaded from disk")

W0807 01:44:50.038103 140724497811264 deprecation_wrapper.py:119] From /home/aditya/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 01:45:04.279694 140724497811264 deprecation_wrapper.py:119] From /home/aditya/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 01:45:04.281400 140724497811264 deprecation_wrapper.py:119] From /home/aditya/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 01:45:04.368346 140724497811264 deprecation_wrapper.py:119] From /home/aditya/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_defa

model loaded from disk
